In [367]:
import warnings
warnings.simplefilter("ignore")

In [368]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load data

olist_customers_dataset

In [369]:
df_customer = pd.read_csv('./dataset/olist_customers_dataset.csv')

In [370]:
df_customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [371]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


olist_geolocation_dataset

In [372]:
df_geo = pd.read_csv('./dataset/olist_geolocation_dataset.csv')

In [373]:
df_geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [374]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


olist_order_items_dataset

In [375]:
df_order_item = pd.read_csv('./dataset/olist_order_items_dataset.csv')

In [376]:
df_order_item.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [377]:
df_order_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [378]:
df_order_item['shipping_limit_date'] = pd.to_datetime(df_order_item['shipping_limit_date'])

olist_order_payments_dataset

In [379]:
df_order_payment = pd.read_csv('./dataset/olist_order_payments_dataset.csv')

In [380]:
df_order_payment.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [381]:
df_order_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


olist_order_reviews_dataset

In [382]:
df_order_review = pd.read_csv('./dataset/olist_order_reviews_dataset.csv')

In [383]:
df_order_review.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [384]:
df_order_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [385]:
df_order_review['review_creation_date'] = pd.to_datetime(df_order_review['review_creation_date'])
df_order_review['review_answer_timestamp'] = pd.to_datetime(df_order_review['review_answer_timestamp'])

olist_orders_dataset

In [386]:
df_order = pd.read_csv('./dataset/olist_orders_dataset.csv')

In [387]:
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [388]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [389]:
df_order['order_purchase_timestamp'] = pd.to_datetime(df_order['order_purchase_timestamp'])
df_order['order_approved_at'] = pd.to_datetime(df_order['order_approved_at'])
df_order['order_delivered_carrier_date'] = pd.to_datetime(df_order['order_delivered_carrier_date'])
df_order['order_delivered_customer_date'] = pd.to_datetime(df_order['order_delivered_customer_date'])
df_order['order_estimated_delivery_date'] = pd.to_datetime(df_order['order_estimated_delivery_date'])

olist_products_dataset

In [390]:
df_product = pd.read_csv('./dataset/olist_products_dataset.csv')

In [391]:
df_product.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [392]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


olist_sellers_dataset

In [393]:
df_seller = pd.read_csv('./dataset/olist_sellers_dataset.csv')

In [394]:
df_seller.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [395]:
df_seller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


product_category_name_translation

In [396]:
df_cat_name = pd.read_csv('./dataset/product_category_name_translation.csv')

In [397]:
df_cat_name.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [398]:
df_cat_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


olist_age

In [399]:
df_age = pd.read_csv('./dataset/olist_age.csv')

In [400]:
df_age.head()

,customer_id,age
0,00012a2ce6f8dcda20d059ce98491703,61
1,000161a058600d5901f007fab4c27140,49
2,0001fd6190edaaf884bcaf3d49edf079,71
3,0002414f95344307404f0ace7a26f1d5,68
4,000379cdec625522490c315e70c7a9fb,61


In [401]:
df_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98666 entries, 0 to 98665
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  98666 non-null  object
 1   age          98666 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


# 2. Feature engineering

1. customer_id

In [402]:
df_customer_unique_id = duckdb.query("""
select distinct customer_unique_id
from df_customer
""").to_df()

In [403]:
df_customer_unique_id.head()

,customer_unique_id
0,345ecd01c38d18a9036ed96c73b8d066
1,9afe194fb833f79e300e37e580171f22
2,7f3a72e8f988c6e735ba118d54f47458
3,a8654e2af5da6bb72f52c22b164855e1
4,07d190f123147d9e89d4b922543d7948


2. master date

In [404]:
duckdb.query("""
select min(order_purchase_timestamp) min_order_purchase_yyyymm
, max(order_purchase_timestamp) max_order_purchase_yyyymm
from df_order
""").to_df()

,min_order_purchase_yyyymm,max_order_purchase_yyyymm
0,2016-09-04 21:15:19,2018-10-17 17:30:18


In [405]:
date_range = pd.date_range(start='2016-09-04', end='2018-10-17', freq='M')

In [406]:
formatted_dates = date_range.strftime('%Y%m')

In [407]:
df_master_date = pd.DataFrame(formatted_dates, columns=['yyyymm'])

In [408]:
df_master_date = duckdb.query("""
select distinct case when cast(right(yyyymm,2) as int) between 1 and 3 then cast(left(yyyymm,4) as int)*100 + 1
when cast(right(yyyymm,2) as int) between 4 and 6 then cast(left(yyyymm,4) as int)*100 + 2
when cast(right(yyyymm,2) as int) between 7 and 9 then cast(left(yyyymm,4) as int)*100 + 3
when cast(right(yyyymm,2) as int) between 10 and 12 then cast(left(yyyymm,4) as int)*100 + 4
end yyyyqq
, yyyymm
from df_master_date
""").to_df()

In [409]:
df_master_date.head()

,yyyyqq,yyyymm
0,201603,201609
1,201701,201702
2,201702,201705
3,201704,201712
4,201702,201706


3. signal

In [410]:
df_signal = duckdb.query("""
select customer_unique_id, yyyymm_signal
, max(case when cat_new = 'home and kitchen' then 1 else 0 end) pur_home
, max(case when cat_new = 'fashion and accessories' then 1 else 0 end) pur_fashion
, max(case when cat_new = 'electronics and gadgets' then 1 else 0 end) pur_elec
, max(case when cat_new = 'tools and construction' then 1 else 0 end) pur_tools
, max(case when cat_new = 'health, beauty, and hygiene' then 1 else 0 end) pur_health_beauty
, max(case when cat_new = 'books, music, and entertainment' then 1 else 0 end) pur_book_and_music
, max(case when cat_new = 'toys and leisure' then 1 else 0 end) pur_toy
, max(case when cat_new = 'miscellaneous' then 1 else 0 end) pur_others
from
    (
    select *
    , year(order_purchase_timestamp)*100 + month(order_purchase_timestamp) yyyymm_signal
    , case when e.product_category_name_english in ('housewares', 'kitchen_dining_laundry_garden_furniture', 'home_confort', 'home_construction', 'home_appliances', 'home_appliances_2', 'bed_bath_table', 'furniture_bedroom', 'furniture_living_room', 'furniture_decor', 'furniture_mattress_and_upholstery', 'small_appliances', 'small_appliances_home_oven_and_coffee', 'garden_tools', 'home_comfort_2') then 'home and kitchen'
    when e.product_category_name_english in ('fashion_bags_accessories', 'fashion_male_clothing', 'fashion_sport', 'fashion_underwear_beach', 'fashion_childrens_clothes', 'fashion_shoes', 'fashio_female_clothing') then 'fashion and accessories'
    when e.product_category_name_english in ('electronics', 'fixed_telephony', 'consoles_games', 'audio', 'tablets_printing_image', 'computers_accessories', 'telephony', 'computers') then 'electronics and gadgets'
    when e.product_category_name_english in ('costruction_tools_garden', 'costruction_tools_tools', 'construction_tools_safety', 'construction_tools_lights', 'construction_tools_construction') then 'tools and construction'
    when e.product_category_name_english in ('health_beauty', 'diapers_and_hygiene') then 'health, beauty, and hygiene'
    when e.product_category_name_english in ('books_general_interest', 'books_imported', 'books_technical', 'music', 'cds_dvds_musicals', 'dvds_blu_ray', 'cine_photo') then 'books, music, and entertainment'
    when e.product_category_name_english in ('baby', 'sports_leisure', 'toys') then 'toys and leisure'
    else 'miscellaneous' end cat_new
    from df_order a
    left join df_customer b
    on a.customer_id = b.customer_id
    left join df_order_item c
    on a.order_id = c.order_id
    left join df_product d
    on c.product_id = d.product_id
    left join df_cat_name e
    on d.product_category_name = e.product_category_name
    where a.order_status not in ('unavailable','canceled')
    ) a
group by customer_unique_id, yyyymm_signal
""").to_df()

In [411]:
df_signal.head()

,customer_unique_id,yyyymm_signal,pur_home,pur_fashion,pur_elec,pur_tools,pur_health_beauty,pur_book_and_music,pur_toy,pur_others
0,21f748a16f4e1688a9014eb3ee6fa325,201801,1,0,0,0,0,0,0,0
1,482441ea6a06b1f72fe9784756c0ea75,201805,0,0,0,0,0,0,1,0
2,c9158d089637ab443c78984d20da7fc0,201801,0,1,0,0,0,0,0,0
3,d4397835ae287e492b186d497099439a,201803,0,0,0,0,1,0,0,0
4,7a380cb5434e6b6b5b37d45bb99dbe8a,201805,1,0,0,0,0,0,0,0


4. age

In [412]:
df_customer_age = duckdb.query("""
select customer_unique_id
, cast(year(birthday)*100 + month(birthday) as int) yyyymm_birthday
from                          
    (
    select customer_unique_id
    , date_add(cast('2018-10-17' as date),-age_day) birthday
    from                           
        (
        select b.customer_unique_id
        , min(a.age) age
        , cast(min(a.age)*365.25 as int) age_day
        from df_age a
        left join df_customer b
        on a.customer_id = b.customer_id
        group by b.customer_unique_id
        ) a
    ) b
""").to_df()

In [413]:
df_customer_age.head()

,customer_unique_id,yyyymm_birthday
0,f90f55ee274a4ae21510b386134b09cd,197910
1,74541fbb7526dabecd0fc96b1192b9a7,199610
2,533b3a42bf5005c302539e093b445d3d,195210
3,4662682dade3cc1bfa04996c5225a849,197710
4,163b27a06a32c2fa565927170b59b5d4,199310


5. customer address

In [414]:
df_customer_address = duckdb.query("""
select customer_unique_id
, case when customer_state in ('AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO') then 1 else 0 end address_north
, case when customer_state in ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') then 1 else 0 end address_northeast
, case when customer_state in ('DF', 'GO', 'MT', 'MS') then 1 else 0 end address_centralwest
, case when customer_state in ('ES', 'MG', 'RJ', 'SP') then 1 else 0 end address_southeast
, case when customer_state in ('PR', 'RS', 'SC') then 1 else 0 end address_south
from
    (
    select customer_unique_id, customer_state, customer_city, customer_zip_code_prefix
    , row_number() over(partition by customer_unique_id order by customer_zip_code_prefix) rn
    from df_customer
    ) a
where rn = 1
""").to_df()

In [415]:
df_customer_address.head()

,customer_unique_id,address_north,address_northeast,address_centralwest,address_southeast,address_south
0,0005ef4cd20d2893f0d9fbd94d3c0d97,0,1,0,0,0
1,000bfa1d2f1a41876493be685390d6d3,0,0,0,1,0
2,001ae44fa04911a9e9577356dce6c63c,0,0,0,1,0
3,003035f5ec76f2220098377482de93c9,0,0,0,1,0
4,004b45ec5c64187465168251cd1c9c2f,0,1,0,0,0


6. number of purchase

In [416]:
df_no_purchase = duckdb.query("""
select customer_unique_id, yyyymm_order
, count(distinct case when cat_new = 'fashion and accessories' then order_id else null end) no_pur_home
, count(distinct case when cat_new = 'electronics and gadgets' then order_id else null end) no_pur_fashion
, count(distinct case when cat_new = 'tools and construction' then order_id else null end) no_pur_tools
, count(distinct case when cat_new = 'health, beauty, and hygiene' then order_id else null end) no_pur_health_beauty
, count(distinct case when cat_new = 'books, music, and entertainment' then order_id else null end) no_pur_book_and_music
, count(distinct case when cat_new = 'toys and leisure' then order_id else null end) no_pur_toy
, count(distinct case when cat_new = 'miscellaneous' then order_id else null end) no_pur_others
from
    (
    select *
    , year(order_purchase_timestamp)*100 + month(order_purchase_timestamp) yyyymm_order
    , case when e.product_category_name_english in ('housewares', 'kitchen_dining_laundry_garden_furniture', 'home_confort', 'home_construction', 'home_appliances', 'home_appliances_2', 'bed_bath_table', 'furniture_bedroom', 'furniture_living_room', 'furniture_decor', 'furniture_mattress_and_upholstery', 'small_appliances', 'small_appliances_home_oven_and_coffee', 'garden_tools', 'home_comfort_2') then 'home and kitchen'
    when e.product_category_name_english in ('fashion_bags_accessories', 'fashion_male_clothing', 'fashion_sport', 'fashion_underwear_beach', 'fashion_childrens_clothes', 'fashion_shoes', 'fashio_female_clothing') then 'fashion and accessories'
    when e.product_category_name_english in ('electronics', 'fixed_telephony', 'consoles_games', 'audio', 'tablets_printing_image', 'computers_accessories', 'telephony', 'computers') then 'electronics and gadgets'
    when e.product_category_name_english in ('costruction_tools_garden', 'costruction_tools_tools', 'construction_tools_safety', 'construction_tools_lights', 'construction_tools_construction') then 'tools and construction'
    when e.product_category_name_english in ('health_beauty', 'diapers_and_hygiene') then 'health, beauty, and hygiene'
    when e.product_category_name_english in ('books_general_interest', 'books_imported', 'books_technical', 'music', 'cds_dvds_musicals', 'dvds_blu_ray', 'cine_photo') then 'books, music, and entertainment'
    when e.product_category_name_english in ('baby', 'sports_leisure', 'toys') then 'toys and leisure'
    else 'miscellaneous' end cat_new
    from df_order a
    left join df_customer b
    on a.customer_id = b.customer_id
    left join df_order_item c
    on a.order_id = c.order_id
    left join df_product d
    on c.product_id = d.product_id
    left join df_cat_name e
    on d.product_category_name = e.product_category_name
    where a.order_status not in ('unavailable','canceled')
    ) a
group by customer_unique_id, yyyymm_order
""").to_df()

In [417]:
df_no_purchase.head()

,customer_unique_id,yyyymm_order,no_pur_home,no_pur_fashion,no_pur_tools,no_pur_health_beauty,no_pur_book_and_music,no_pur_toy,no_pur_others
0,9dbb05f5577e862337b93feb8f358839,201808,0,0,0,1,0,0,0
1,0e0d3aed77ac2e3746c2ddc3101e3d43,201708,0,0,0,0,0,0,0
2,005226be8d5b972d0f72b4e8a79f322b,201804,0,0,0,0,0,0,0
3,c3ae72f7fe12701be16e17345697d359,201711,0,0,0,0,0,1,0
4,223ab530d4126650969ce8bc7370a21b,201801,0,0,0,0,0,0,0


7. amount purchase

In [418]:
df_amt_purchase = duckdb.query("""
select customer_unique_id, yyyymm_order
, sum(case when cat_new = 'fashion and accessories' then price_new + freight_value_new else 0 end) amt_pur_home
, sum(case when cat_new = 'electronics and gadgets' then price_new + freight_value_new else 0 end) amt_pur_fashion
, sum(case when cat_new = 'tools and construction' then price_new + freight_value_new else 0 end) amt_pur_tools
, sum(case when cat_new = 'health, beauty, and hygiene' then price_new + freight_value_new else 0 end) amt_pur_health_beauty
, sum(case when cat_new = 'books, music, and entertainment' then price_new + freight_value_new else 0 end) amt_pur_book_and_music
, sum(case when cat_new = 'toys and leisure' then price_new + freight_value_new else 0 end) amt_pur_toy
, sum(case when cat_new = 'miscellaneous' then price_new + freight_value_new else 0 end) amt_pur_others
from
    (
    select *
    , year(order_purchase_timestamp)*100 + month(order_purchase_timestamp) yyyymm_order
    , case when e.product_category_name_english in ('housewares', 'kitchen_dining_laundry_garden_furniture', 'home_confort', 'home_construction', 'home_appliances', 'home_appliances_2', 'bed_bath_table', 'furniture_bedroom', 'furniture_living_room', 'furniture_decor', 'furniture_mattress_and_upholstery', 'small_appliances', 'small_appliances_home_oven_and_coffee', 'garden_tools', 'home_comfort_2') then 'home and kitchen'
    when e.product_category_name_english in ('fashion_bags_accessories', 'fashion_male_clothing', 'fashion_sport', 'fashion_underwear_beach', 'fashion_childrens_clothes', 'fashion_shoes', 'fashio_female_clothing') then 'fashion and accessories'
    when e.product_category_name_english in ('electronics', 'fixed_telephony', 'consoles_games', 'audio', 'tablets_printing_image', 'computers_accessories', 'telephony', 'computers') then 'electronics and gadgets'
    when e.product_category_name_english in ('costruction_tools_garden', 'costruction_tools_tools', 'construction_tools_safety', 'construction_tools_lights', 'construction_tools_construction') then 'tools and construction'
    when e.product_category_name_english in ('health_beauty', 'diapers_and_hygiene') then 'health, beauty, and hygiene'
    when e.product_category_name_english in ('books_general_interest', 'books_imported', 'books_technical', 'music', 'cds_dvds_musicals', 'dvds_blu_ray', 'cine_photo') then 'books, music, and entertainment'
    when e.product_category_name_english in ('baby', 'sports_leisure', 'toys') then 'toys and leisure'
    else 'miscellaneous' end cat_new
    , case when price is null then 0 else price end price_new
    , case when freight_value is null then 0 else 0 end freight_value_new
    from df_order a
    left join df_customer b
    on a.customer_id = b.customer_id
    left join df_order_item c
    on a.order_id = c.order_id
    left join df_product d
    on c.product_id = d.product_id
    left join df_cat_name e
    on d.product_category_name = e.product_category_name
    where a.order_status not in ('unavailable','canceled')
    ) a
group by customer_unique_id, yyyymm_order
""").to_df()

In [419]:
df_amt_purchase.head()

,customer_unique_id,yyyymm_order,amt_pur_home,amt_pur_fashion,amt_pur_tools,amt_pur_health_beauty,amt_pur_book_and_music,amt_pur_toy,amt_pur_others
0,2a46fb94aef5cbeeb850418118cee090,201802,0.0,129.9,0.0,0.00,0.0,0.0,0.00
1,918dc87cd72cd9f6ed4bd442ed785235,201709,0.0,0.0,0.0,0.00,0.0,0.0,99.00
2,cd076285a12f40041b32f5ad8c98699f,201703,0.0,0.0,0.0,0.00,0.0,0.0,29.98
3,67d21c8bea9d6017d1b124d3879dd815,201803,0.0,0.0,0.0,0.00,0.0,0.0,120.00
4,9dbb05f5577e862337b93feb8f358839,201808,0.0,0.0,0.0,66.99,0.0,0.0,0.00


8. payment channel

In [420]:
df_payment_channel = duckdb.query("""
select customer_unique_id
, yyyymm_order
, count(distinct case when payment_type = 'credit_card' then order_id else null end) payment_credit
, count(distinct case when payment_type = 'debit_card' then order_id else null end) payment_debit
, count(distinct case when payment_type = 'boleto' then order_id else null end) payment_ticket
, count(distinct case when payment_type = 'voucher' then order_id else null end) payment_voucher
from
    (
    select *
    , year(order_purchase_timestamp)*100 + month(order_purchase_timestamp) yyyymm_order                                             
    from df_order a
    left join df_customer b
    on a.customer_id = b.customer_id
    left join df_order_payment c
    on a.order_id = c.order_id
    where a.order_status not in ('unavailable','canceled')
    ) a
group by customer_unique_id, yyyymm_order
""").to_df()

In [421]:
df_payment_channel.head()

,customer_unique_id,yyyymm_order,payment_credit,payment_debit,payment_ticket,payment_voucher
0,f1a7919983d547880b4111fd4f77c74d,201804,1,0,0,0
1,58e1140fb8f15f1a010d01a34bfbf77d,201802,1,0,0,0
2,691d75928efd2c5dd34b678f19783c28,201802,1,0,0,0
3,20bb1d9431264f331e47b2da94991eaf,201808,0,0,1,0
4,5e21109503a635f7ad93db18a7ee4a21,201710,0,0,1,0


9. merge

In [422]:
df_final = duckdb.query("""
select a.customer_unique_id, b.yyyymm, b.yyyyqq
, pur_home, pur_fashion, pur_elec, pur_tools, pur_health_beauty, pur_book_and_music, pur_toy, pur_others
, ((cast(b.yyyymm as int)//100)*12+(cast(b.yyyymm as int)%100)) - ((yyyymm_birthday//100)*12+(yyyymm_birthday%100)) age_month
, address_north, address_northeast, address_centralwest, address_southeast, address_south
, no_pur_home, no_pur_fashion, no_pur_tools, no_pur_health_beauty, no_pur_book_and_music, no_pur_toy, no_pur_others
, amt_pur_home, amt_pur_fashion, amt_pur_tools, amt_pur_health_beauty, amt_pur_book_and_music, amt_pur_toy, amt_pur_others
, payment_credit, payment_debit, payment_ticket, payment_voucher
from df_customer_unique_id a        
left join df_master_date b
on 1=1
left join df_signal c
on a.customer_unique_id = c.customer_unique_id
and b.yyyymm = c.yyyymm_signal          
left join df_customer_age d
on a.customer_unique_id = d.customer_unique_id
left join df_customer_address e
on a.customer_unique_id = e.customer_unique_id
left join df_no_purchase f
on a.customer_unique_id = f.customer_unique_id
and b.yyyymm = f.yyyymm_order
left join df_amt_purchase g
on a.customer_unique_id = g.customer_unique_id
and b.yyyymm = g.yyyymm_order            
left join df_payment_channel h
on a.customer_unique_id = h.customer_unique_id
and b.yyyymm = h.yyyymm_order   
where d.yyyymm_birthday is not null
""").to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [423]:
df_final = df_final.fillna(0)

In [428]:
df_final = duckdb.query("""                    
select customer_unique_id, yyyyqq
, max(pur_home) as pur_home
, max(pur_fashion) as pur_fashion
, max(pur_elec) as pur_elec
, max(pur_tools) as pur_tools
, max(pur_health_beauty) as pur_health_beauty
, max(pur_book_and_music) as pur_book_and_music
, max(pur_toy) as pur_toy
, max(pur_others) as pur_others
, avg(age_month) as age_month
, max(address_north) as address_north
, max(address_northeast) as address_northeast
, max(address_centralwest) as address_centralwest
, max(address_southeast) as address_southeast
, max(address_south) as address_south
, sum(no_pur_home) as no_pur_home
, sum(no_pur_fashion) as no_pur_fashion
, sum(no_pur_tools) as no_pur_tools
, sum(no_pur_health_beauty) as no_pur_health_beauty
, sum(no_pur_book_and_music) as no_pur_book_and_music
, sum(no_pur_toy) as no_pur_toy
, sum(no_pur_others) as no_pur_others
, sum(amt_pur_home) as amt_pur_home
, sum(amt_pur_fashion) as amt_pur_fashion
, sum(amt_pur_tools) as amt_pur_tools
, sum(amt_pur_health_beauty) as amt_pur_health_beauty
, sum(amt_pur_book_and_music) as amt_pur_book_and_music
, sum(amt_pur_toy) as amt_pur_toy
, sum(amt_pur_others) as amt_pur_others
, sum(payment_credit) as payment_credit
, sum(payment_debit) as payment_debit
, sum(payment_ticket) as payment_ticket
, sum(payment_voucher) as payment_voucher
from
    (
    select *
    from df_final
    ) a
where customer_unique_id = 'bc75c925b9f7994e23fe4ecd6f1004f8'                        
group by customer_unique_id, yyyyqq
""").to_df()

In [432]:
df_final = duckdb.query("""    
select *
, lead(pur_home) over (partition by customer_unique_id order by yyyyqq) as lead_pur_home
, lead(pur_fashion) over (partition by customer_unique_id order by yyyyqq) as lead_pur_fashion
, lead(pur_elec) over (partition by customer_unique_id order by yyyyqq) as lead_pur_elec
, lead(pur_tools) over (partition by customer_unique_id order by yyyyqq) as lead_pur_tools
, lead(pur_health_beauty) over (partition by customer_unique_id order by yyyyqq) as lead_pur_health_beauty
, lead(pur_book_and_music) over (partition by customer_unique_id order by yyyyqq) as lead_pur_book_and_music
, lead(pur_toy) over (partition by customer_unique_id order by yyyyqq) as lead_pur_toy
, lead(pur_others) over (partition by customer_unique_id order by yyyyqq) as lead_pur_others
from
    (
    select *
    from df_final
    ) a
where customer_unique_id = 'bc75c925b9f7994e23fe4ecd6f1004f8'
""").to_df()

In [433]:
df_final

,customer_unique_id,yyyyqq,pur_home,pur_fashion,pur_elec,pur_tools,pur_health_beauty,pur_book_and_music,pur_toy,pur_others,...,lead_pur_toy,lead_pur_others,lead_pur_home_1,lead_pur_fashion_1,lead_pur_elec_1,lead_pur_tools_1,lead_pur_health_beauty_1,lead_pur_book_and_music_1,lead_pur_toy_1,lead_pur_others_1
0,bc75c925b9f7994e23fe4ecd6f1004f8,201603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bc75c925b9f7994e23fe4ecd6f1004f8,201604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bc75c925b9f7994e23fe4ecd6f1004f8,201701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bc75c925b9f7994e23fe4ecd6f1004f8,201702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bc75c925b9f7994e23fe4ecd6f1004f8,201703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,bc75c925b9f7994e23fe4ecd6f1004f8,201704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,bc75c925b9f7994e23fe4ecd6f1004f8,201801,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,bc75c925b9f7994e23fe4ecd6f1004f8,201802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,bc75c925b9f7994e23fe4ecd6f1004f8,201803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [426]:
**
    AVG(A) OVER (PARTITION BY customer_unique_id ORDER BY yyyyqq ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS avg_3_quarter_A


SyntaxError: invalid syntax (402238255.py, line 1)

10. create feature

In [435]:
duckdb.query("""
select customer_unique_id, yyyyqq, 
from df_final
where customer_unique_id = 'bc75c925b9f7994e23fe4ecd6f1004f8'
""").to_df()

,customer_unique_id,yyyyqq,pur_home,pur_fashion,pur_elec,pur_tools,pur_health_beauty,pur_book_and_music,pur_toy,pur_others,...,lead_pur_toy,lead_pur_others,lead_pur_home_1,lead_pur_fashion_1,lead_pur_elec_1,lead_pur_tools_1,lead_pur_health_beauty_1,lead_pur_book_and_music_1,lead_pur_toy_1,lead_pur_others_1
0,bc75c925b9f7994e23fe4ecd6f1004f8,201603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bc75c925b9f7994e23fe4ecd6f1004f8,201604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bc75c925b9f7994e23fe4ecd6f1004f8,201701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bc75c925b9f7994e23fe4ecd6f1004f8,201702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bc75c925b9f7994e23fe4ecd6f1004f8,201703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,bc75c925b9f7994e23fe4ecd6f1004f8,201704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,bc75c925b9f7994e23fe4ecd6f1004f8,201801,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,bc75c925b9f7994e23fe4ecd6f1004f8,201802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,bc75c925b9f7994e23fe4ecd6f1004f8,201803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
duckdb.query("""
select *
from
    (
    select *
    , case when cast(right(yyyymm,2) as int) between 1 and 3 then cast(left(yyyymm,4) as int)*100 + 1
    when cast(right(yyyymm,2) as int) between 4 and 6 then cast(left(yyyymm,4) as int)*100 + 2
    when cast(right(yyyymm,2) as int) between 7 and 9 then cast(left(yyyymm,4) as int)*100 + 3
    when cast(right(yyyymm,2) as int) between 10 and 12 then cast(left(yyyymm,4) as int)*100 + 4
    end yyyyqq
    from df_final
    where customer_unique_id = 'f532b515dbddb3fcc2c63367244dac4b'
    ) a
""").to_df()

In [ ]:
***

In [ ]:
import pandas as pd

# Ensure DataFrame is sorted by customer ID and date
df_final = df_final.sort_values(by=['customer_unique_id', 'yyyymm'])

# Define purchase amount columns
purchase_amount_columns = ['amt_pur_home', 'amt_pur_fashion', 'amt_pur_tools', 'amt_pur_health_beauty', 'amt_pur_book_and_music', 'amt_pur_toy', 'amt_pur_others']

# Calculate 3-month moving average for each purchase amount column
for col in purchase_amount_columns:
    df_final[f'avg_m3_{col}'] = df_final.groupby('customer_unique_id')[col].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Display the engineered features
import ace_tools as tools; tools.display_dataframe_to_user(name="Lag Features DataFrame", dataframe=df_final)

# Display the DataFrame with new lag features
df_final.head()

# 5. Exploratory data analysis (EDA)

list of columns by types

In [ ]:
df.columns.tolist()

In [ ]:
id_col = ['customer_unique_id']

In [ ]:
cat_col = ['address_north','address_northeast','address_centralwest','address_southeast','address_south']

In [ ]:
num_col = ['age',
           'day_after_last_pur','no_order','total_amount',
           'cat_home','cat_fashion','cat_elec','cat_tools','cat_health_beauty',
           'cat_book_and_music','cat_toy','cat_others',
           'payment_credit','payment_debit','payment_ticket','payment_voucher',
           'avg_review_score']

categorical features

In [ ]:
df[cat_col].describe()

numeric feature

In [ ]:
for column_name in num_col:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.hist(df[column_name], bins=10, edgecolor='black', alpha=0.7)
    ax1.set_title(f'Histogram of {column_name}')
    ax1.set_xlabel(column_name)
    ax1.set_ylabel('Frequency')
    ax1.grid(axis='y', alpha=0.75)
    
    if isinstance(df[column_name].dtype, pd.CategoricalDtype):
        sns.boxplot(x=df[column_name], ax=ax2)
    else:
        sns.boxplot(x=df[column_name], ax=ax2)
    ax2.set_title(f'Boxplot of {column_name} (No Hue)')
    ax2.set_xlabel(column_name)
    
    plt.show()

drop unnecessary columns

In [ ]:
df.drop(columns = 'customer_unique_id', inplace = True)

correlation

In [ ]:
corr = df.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(corr, annot=True, ax=ax)

# 6. Model

In [ ]:
df.head()

standardize 

In [ ]:
df = df[cat_col + num_col]

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(df)

In [ ]:
df_scaled = pd.DataFrame(scaler.transform(df), columns=cat_col + num_col)

In [ ]:
df

In [ ]:
df_scaled

PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
df_pca = pd.DataFrame(data=pca.fit_transform(df_scaled), columns=['Principal Component 1', 'Principal Component 2'])

In [ ]:
df_pca

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c='blue', label='Data Points')

plt.title('PCA Results with 2 Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()

plt.show()

elbow method

In [ ]:
range_n_clusters = range(2, 11)

In [ ]:
wcss = []

In [ ]:
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(df_pca)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range_n_clusters, wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

silhouette

In [ ]:
silhouette_scores = []

In [ ]:
for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(df_pca)
    silhouette_avg = silhouette_score(df_pca, cluster_labels)
    silhouette_scores.append(silhouette_avg)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Silhouette Scores')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

KMeans (k=4)

In [ ]:
kmeans_k4 = KMeans(n_clusters=4, random_state=42)

In [ ]:
kmeans_k4.fit(df_pca)

In [ ]:
clusters_kmean_k4 = kmeans_k4.predict(df_pca)

In [ ]:
clusters_kmean_k4

In [ ]:
df_pca['cluster (Kmean k=4)'] = clusters_kmean_k4
df['cluster (Kmean k=4)'] = clusters_kmean_k4

KMeans (k=6)

In [ ]:
kmeans_k6 = KMeans(n_clusters=6, random_state=42)

In [ ]:
kmeans_k6.fit(df_pca)

In [ ]:
clusters_kmean_k6 = kmeans_k6.predict(df_pca)

In [ ]:
clusters_kmean_k6

In [ ]:
df_pca['cluster (Kmean k=6)'] = clusters_kmean_k6
df['cluster (Kmean k=6)'] = clusters_kmean_k6

Plot

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c=df_pca['cluster (Kmean k=4)'], cmap='viridis', label='Cluster Label')

plt.title('PCA Results with 2 Components and K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.show()

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c=df_pca['cluster (Kmean k=6)'], cmap='viridis', label='Cluster Label')

plt.title('PCA Results with 2 Components and K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.show()

Average value by cluster

In [ ]:
df.groupby('cluster (Kmean k=4)').mean().T

In [ ]:
df.groupby('cluster (Kmean k=6)').mean().T

# Save model

In [ ]:
with open('scaler_model.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

In [ ]:
# Save the KMeans model
with open('kmeans_model.pkl', 'wb') as kmeans_file:
    pickle.dump(kmeans_k4, kmeans_file)